In [ ]:
import os, glob
import micasense.capture as capture
%matplotlib inline

panelNames = None

# # This is an older RedEdge image without RigRelatives
# imagePath = os.path.join(os.path.abspath('.'),'data','0000SET','000')
# imageNames = glob.glob(os.path.join(imagePath,'IMG_0001_*.tif'))
# panelNames = glob.glob(os.path.join(imagePath,'IMG_0000_*.tif'))



# # Image from the example RedEdge imageSet (see the ImageSet notebook) without RigRelatives.
# imagePath = os.path.expanduser(os.path.join('/home/cvg-ws05/msi_up/nikhil/MSI/data/DataCapture/agri_data_30jul','0021SET'))
# imageNames = glob.glob(os.path.join(imagePath,'000','IMG_0005_*.tif'))  # sets to be aligned
# panelNames = glob.glob(os.path.join(imagePath,'000','IMG_0000_*.tif'))  # corresponding CRP images for metadata


# Image from the example RedEdge imageSet (see the ImageSet notebook) without RigRelatives.
imagePath = os.path.expanduser(os.path.join('/home/cvg-ws05/msi_up/nikhil/MSI/data/micasense_datasets/Part1','000'))
imageNames = glob.glob(os.path.join(imagePath,'IMG_0077_*.tif'))  # sets to be aligned
panelNames = glob.glob(os.path.join(imagePath,'IMG_0000_*.tif'))  # corresponding CRP images for metadata


# This is an altum image with RigRelatives and a thermal band
# imagePath = os.path.join('.','data','ALTUM1SET','000')
# imageNames = glob.glob(os.path.join(imagePath,'IMG_0245_*.tif'))
# panelNames = glob.glob(os.path.join(imagePath,'IMG_0000_*.tif'))

export_channels = True
channel_path = '/home/cvg-ws05/msi_up/nikhil/MSI/data/micasense_datasets/Part1/channels'  # to save registered MS channels


if export_channels and not os.path.exists(channel_path):
    os.makedirs(channel_path)


# Allow this code to align both radiance and reflectance images; bu excluding
# a definition for panelNames above, radiance images will be used
# For panel images, efforts will be made to automatically extract the panel information
# but if the panel/firmware is before Altum 1.3.5, RedEdge 5.1.7 the panel reflectance
# will need to be set in the panel_reflectance_by_band variable.
# Note: radiance images will not be used to properly create NDVI/NDRE images below.
if panelNames is not None:
    panelCap = capture.Capture.from_filelist(panelNames)

else:
    panelCap = None


capture = capture.Capture.from_filelist(imageNames)

if panelCap is not None:
    if panelCap.panel_albedo() is not None:
        panel_reflectance_by_band = panelCap.panel_albedo()
    else:
        panel_reflectance_by_band = [0.67, 0.69, 0.68, 0.61, 0.67] #RedEdge band_index order
    panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)    
    img_type = "reflectance"
    capture.plot_undistorted_reflectance(panel_irradiance)
else:
    if capture.dls_present():
        img_type='reflectance'
        capture.plot_undistorted_reflectance(capture.dls_irradiance())
    else:
        img_type = "radiance"
        capture.plot_undistorted_radiance() 


print(imagePath)
print(imageNames)   

# Load the warp matrices

In [ ]:
from numpy import array
from numpy import float32
import numpy as np

warp_matrices = np.load('/home/cvg-ws05/msi_up/nikhil/MSI/data/micasense_datasets/Part1/warp_matrices.npy')
# warp_matrices = np.load('../MSI/data/DataCapture/calibration.npy')
print(warp_matrices)
print("[INFO] warp matrices loaded...")

In [ ]:
try:
    irradiance = panel_irradiance+[0]
except NameError:
    irradiance = None


    
sample = capture.create_aligned_capture(irradiance_list=irradiance, warp_matrices=warp_matrices)
print("[INFO] Multispectral channels registered...")

In [ ]:
print(imageNames[0])

# Saving the aligned channels

In [ ]:
print(sample.shape)
import matplotlib.pyplot as plt
for i in range(0, 5):
    plt.imshow(sample[:,:,i])
    plt.savefig('chnl_{}.png'.format(i))
print("[INFO] channels saved...")